In [35]:
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn import set_config
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.tree import DecisionTreeClassifier

import warnings
warnings.filterwarnings("ignore")

In [2]:
df = pd.read_csv('E:/Python Programs/titanic.csv')

In [3]:
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,0,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,0,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,0,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,1,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [4]:
df.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin'], inplace=True)

In [5]:
df.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
0,0,3,male,34.5,0,0,7.8292,Q
1,1,3,female,47.0,1,0,7.0000,S
2,0,2,male,62.0,0,0,9.6875,Q
3,0,3,male,27.0,0,0,8.6625,S
4,1,3,female,22.0,1,1,12.2875,S


In [6]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(columns=['Survived']), df['Survived'],test_size=0.2,random_state=42)
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((334, 7), (334,), (84, 7), (84,))

In [7]:
X_train.head()

,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked
336,2,male,32.0,0,0,13.0000,S
31,2,male,24.0,2,0,31.5000,S
84,2,male,NaN,0,0,10.7083,Q
287,1,male,24.0,1,0,82.2667,S
317,2,male,19.0,0,0,10.5000,S


In [8]:
y_train.sample(5)

345    1
7      0
283    1
213    1
194    0
Name: Survived, dtype: int64

## Imputation Transformer

In [10]:
trf1 = ColumnTransformer([
    ('impute_age',SimpleImputer(),[2]),
    ('impute_Fare',SimpleImputer(),[5]),
    ('impute_embarked',SimpleImputer(strategy='most_frequent'),[6])
],remainder='passthrough')

## One Hot Encoding Transformer on Sex & Embarked columns, both at the same time

In [11]:
trf2 = ColumnTransformer([
    ('ohe_sex_embarked',OneHotEncoder(sparse=False, handle_unknown='ignore'),[1,6])
],remainder='passthrough')

## Scaling Transformer

In [12]:
# remember, after OneHot encoding you will have 10 columns in total.

trf3 = ColumnTransformer([
    ('scale', MinMaxScaler(),slice(0,10))
])

## Feature Selection Transformer

In [13]:
trf4 = SelectKBest(score_func=chi2, k=8)

## Model Training Transformer

In [14]:
# train the model

trf5 = DecisionTreeClassifier()

## Creating Pipeline

In [15]:
pipe = Pipeline([
    ('trf1',trf1),
    ('trf2',trf2),
    ('trf3',trf3),
    ('trf4',trf4),
    ('trf5',trf5),
])

# Pipeline vs make_pipeline

##### Pipeline requires naming of steps, where as make_pipeline does not.
##### Same applies to ColumnTransformer vs make_column_transformer 

In [ ]:
# Alternate Syntax

pipe = make_pipeline(trf1,trf2,trf3,trf4,trf5)

## Display Pipeline

In [18]:
set_config(display='diagram')

In [19]:
# train

pipe.fit(X_train, y_train)

Pipeline(steps=[('trf1',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('impute_age', SimpleImputer(),
                                                  [2]),
                                                 ('impute_Fare',
                                                  SimpleImputer(), [5]),
                                                 ('impute_embarked',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  [6])])),
                ('trf2',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('ohe_sex_embarked',
                                                  OneHotEncoder(handle_unknown='ignore',
                                                                sparse=False),
                                                  [1, 6])])),
                ('trf3',
                 ColumnTransformer(transformers=[('scale', MinMaxScaler(),
                                                  slice(0, 10, None))])),
                ('trf4',
                 SelectKBest(k=8,
                             score_func=<function chi2 at 0x00000202144558A0>)),
                ('trf5', DecisionTreeClassifier())])

## Exploring the Pipeline

In [20]:
pipe.named_steps

{'trf1': ColumnTransformer(remainder='passthrough',
                   transformers=[('impute_age', SimpleImputer(), [2]),
                                 ('impute_Fare', SimpleImputer(), [5]),
                                 ('impute_embarked',
                                  SimpleImputer(strategy='most_frequent'),
                                  [6])]),
 'trf2': ColumnTransformer(remainder='passthrough',
                   transformers=[('ohe_sex_embarked',
                                  OneHotEncoder(handle_unknown='ignore',
                                                sparse=False),
                                  [1, 6])]),
 'trf3': ColumnTransformer(transformers=[('scale', MinMaxScaler(), slice(0, 10, None))]),
 'trf4': SelectKBest(k=8, score_func=<function chi2 at 0x00000202144558A0>),
 'trf5': DecisionTreeClassifier()}

In [21]:
pipe.named_steps['trf1'].transformers_

[('impute_age', SimpleImputer(), [2]),
 ('impute_Fare', SimpleImputer(), [5]),
 ('impute_embarked', SimpleImputer(strategy='most_frequent'), [6]),
 ('remainder', 'passthrough', [0, 1, 3, 4])]

In [22]:
pipe.named_steps['trf1'].transformers_[0]

('impute_age', SimpleImputer(), [2])

In [23]:
pipe.named_steps['trf1'].transformers_[0][1]

SimpleImputer()

In [27]:
# mean value, which SimpleImputer has assigned to missing values in Age column

pipe.named_steps['trf1'].transformers_[0][1].statistics_

array([30.11576336])

In [26]:
# mean value, which SimpleImputer has assigned to missing values in Fare column

pipe.named_steps['trf1'].transformers_[1][1].statistics_

array([36.90913514])

In [28]:
# most frequent value, which SimpleImputer has assigned to missing values in Embarked column

pipe.named_steps['trf1'].transformers_[2][1].statistics_

array(['S'], dtype=object)

## Predict

In [31]:
y_pred = pipe.predict(X_test)
y_pred

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=int64)

In [32]:
accuracy_score(y_test, y_pred)

0.5952380952380952

## Cross Validation using Pipeline

In [34]:
cross_val_score(pipe, X_train, y_train, cv=5, scoring='accuracy').mean()

0.6437358661239257

## GridSearch (i.e., Hyper Parameter Tuning) using Pipeline

In [36]:
# gridsearchcv

params = {
    'trf5__max_depth':[1,2,3,4,5,None]
}

In [37]:
grid = GridSearchCV(pipe, params, cv=5, scoring='accuracy')
grid.fit(X_train, y_train)

GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('trf1',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('impute_age',
                                                                         SimpleImputer(),
                                                                         [2]),
                                                                        ('impute_Fare',
                                                                         SimpleImputer(),
                                                                         [5]),
                                                                        ('impute_embarked',
                                                                         SimpleImputer(strategy='most_frequent'),
                                                                         [6])])),
                                       ('trf2',
                                        ColumnTransformer(remainder='passthrough',
                                                          transformers=[('ohe_sex_embarked',
                                                                         OneHotEncoder(handle_unknown='ignore',
                                                                                       sparse=False),
                                                                         [1,
                                                                          6])])),
                                       ('trf3',
                                        ColumnTransformer(transformers=[('scale',
                                                                         MinMaxScaler(),
                                                                         slice(0, 10, None))])),
                                       ('trf4',
                                        SelectKBest(k=8,
                                                    score_func=<function chi2 at 0x00000202144558A0>)),
                                       ('trf5', DecisionTreeClassifier())]),
             param_grid={'trf5__max_depth': [1, 2, 3, 4, 5, None]},
             scoring='accuracy')

In [38]:
grid.best_score_

0.6467209407507915

In [39]:
grid.best_params_

{'trf5__max_depth': 1}

## Exporting the Pipeline

In [40]:
# export

pickle.dump(pipe, open('models/pipe.pkl','wb'))